In [1]:
# Analyse supply chain data from kaggle
# Source: https://www.kaggle.com/abdelrahmancae/supply-chain-analysis-and-modeling

In [77]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import re

In [6]:
# Import dataset
df = pd.read_csv("SCMS_Delivery_History_Dataset.csv")

In [10]:
df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN


In [208]:
# Create utility functions for data cleaning

# Determine column headers
def printColumns(df):
    columns = list(df.columns)
    return print(f"Columns are {','.join(list(df.columns))}")

# Determine data type for each column
def dataType(df):
    columns = list(df.columns)
    dtypes = [str(df[x].dtype) for x in columns]
    columnType = list(zip(columns, dtypes))
    columnDf = pd.DataFrame(columnType, columns = ['Column', 'Dtype'])
    print(columnDf)
    return columnDf, columnType

# Determine if there exists date columns and convert their data type to datetime if not datetime type
def dateColumns(df, columnType):
    df = df.copy()
    pattern = re.compile(r'^\d')
    dateList = []
    dateType = []
    for i in [x for x in columnType]:
        if 'date' in str(i[0]).lower():
            if str(type(i[1])) != 'datetime.date':
                value = df[i[0]].mode().values[0]
                try: 
                    flag = pattern.match(str(value)).group()
                    columnName = i[0] + '_datetime'
                    df[columnName] = df[i[0]].apply(lambda x: datetime.strptime(x, '%d-%b-%y'))
                    print(f"Added column {columnName} as datetime type.")
                except:
                    pass
    return df         

def costColumns(df, columnType):
    df = df.copy()
    pattern = re.compile(r'^\d')
    costList = []
    costType = []
    for i in [x for x in columnType]:
        if 'cost' in str(i[0]).lower():
            if str(type(i[1])) != 'float':
                value = df[i[0]].mode().values[0]
                try: 
                    flag = pattern.match(str(value))
                    columnName = i[0] + '_float'
                    df[columnName] = df[i[0]].apply(lambda x: float(x) if pattern.match(x) else 0)
                    print(f"Added column {columnName} as float type.")
                except:
                    pass
    return df         

def cleanPipeline(df):
    printColumns(df)
    columnDf, columnType = dataType(df)
    df = dateColumns(df, columnType)
    df = costColumns(df, columnType)
    return df

In [209]:
df = cleanPipeline(df)

Columns are ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,PO Sent to Vendor Date,Scheduled Delivery Date,Delivered to Client Date,Delivery Recorded Date,Product Group,Sub Classification,Vendor,Item Description,Molecule/Test Type,Brand,Dosage,Dosage Form,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD),Scheduled Delivery Date_datetime,Delivered to Client Date_datetime,Delivery Recorded Date_datetime,isLate
                               Column           Dtype
0                                  ID           int64
1                        Project Code          object
2                                PQ #          object
3                           PO / SO #          object
4                            ASN/DN #          object
5                             Country    

In [220]:
# Create functions to add features
def lateDelivery(df):
    df = df.copy()
    df['isLate'] = df['Delivered to Client Date_datetime'] - df['Scheduled Delivery Date_datetime']
    df['isLate'] = df['isLate'].apply(lambda x: 1 if x.days > 0 else 0)
    return df

def madeProfit(df):
    df = df.copy()
    df['Profit'] = df['Line Item Value'] - (df['Line Item Insurance (USD)'].fillna(0) + df['Freight Cost (USD)_float'])
    df['isLoss'] = df['Profit'].apply(lambda x: 1 if x < 0 else 0)
    return df

def featuresPipeline(df):
    df = lateDelivery(df)
    df = madeProfit(df)
    return df

In [221]:
df = featuresPipeline(df)

In [222]:
df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD),Scheduled Delivery Date_datetime,Delivered to Client Date_datetime,Delivery Recorded Date_datetime,isLate,Freight Cost (USD)_float,Profit,isLoss
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,13,780.34,NaN,2006-06-02,2006-06-02,2006-06-02,0,780.34,-229.34,1
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,358,4521.5,NaN,2006-11-14,2006-11-14,2006-11-14,0,4521.50,1678.50,0
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,171,1653.78,NaN,2006-08-27,2006-08-27,2006-08-27,0,1653.78,38346.22,0
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,1855,16007.06,NaN,2006-09-01,2006-09-01,2006-09-01,0,16007.06,111353.74,0
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,7590,45450.08,NaN,2006-08-11,2006-08-11,2006-08-11,0,45450.08,76149.92,0
